# Example 5: Quantum-to-quantum transfer learning.

This is an example of a continuous variable (CV) quantum network for state classification, developed according to the *quantum-to-quantum transfer learning* scheme presented in [1]. 

## Introduction 


In this proof-of-principle demonstration we consider two distinct toy datasets of Gaussian and non-Gaussian states. Such datasets can be generated according to the following simple prescriptions:

**Dataset A**:  
- Class 0 (Gaussian): random Gaussian layer applied to the vacuum.  
- Class 1 (non-Gaussian): random non-Gaussian Layer applied to the vacuum.  
        
**Dataset B**:  
- Class 0 (Gaussian): random Gaussian layer applied to a coherent state with amplitude $\alpha=1$.
- Class 1 (non-Gaussian): random Gaussian layer applied to a single photon Fock state $|1\rangle$.

**Variational Circuit A**:  
Our starting point is a single-mode variational circuit [2] (a non-Gaussian layer), pre-trained on _Dataset A_. We assume that after the circuit is applied, the output mode is measured with an _on/off_ detector. By averaging over many shots, one can estimate the vacuum probability:

$$
p_0 = | \langle \psi_{\rm out} |0 \rangle|^2. 
$$

We use _Dataset A_ and train the circuit to rotate Gaussian states towards the vacuum while non-Gaussian states far away from the vacuum. For the final classification we use the simple decision rule:

$$
p_0 \ge 0  \longrightarrow {\rm Class=0.} \\
p_0 < 0  \longrightarrow {\rm Class=1.}
$$

**Variational Circuit B**:  
Once _Circuit A_ has been optimized, we can use is as a pre-trained block
applicable also to the different _Dataset B_. In other words, we implement a _quantum-to-quantum_ transfer learning model:

_Circuit B_ =  _Circuit A_ (pre-trained) followed by a sequence of  _variational layers_ (to be trained).

Also in this case, after the application of _Circuit B_, we assume to measure the single mode with an _on/off_ detector, and we apply a similar classification rule:

$$
p_0 \ge 0  \longrightarrow {\rm Class=1.} \\
p_0 < 0  \longrightarrow {\rm Class=0.}
$$

The motivation for this transfer learning approach is that, even if _Circuit A_ is optimized on a different dataset, it can still act as a good pre-processing block also for _Dataset B_. Ineeed, as we are going to show, the application of _Circuit A_ can significantly improve the training efficiency of _Circuit B_.

## General setup

The main imported modules are: the `tensorflow` machine learning framework, the quantum CV 
software `strawberryfields` [3] and the python plotting library `matplotlib`. All modules should be correctly installed in the system before running this notebook.

In [2]:
# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf

# Strawberryfields (simulation of CV quantum circuits)
import strawberryfields as sf
from strawberryfields.ops import Dgate, Kgate, Sgate, Rgate, Vgate, Fock, Ket

# Other modules
import numpy as np
import time

# System variables
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # avoid warning messages
os.environ['OMP_NUM_THREADS'] = '1'       # set number of threads.
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # select the GPU unit.

# Path with pre-trained parameters
weights_path = 'results/weights/'

Setting of the main parameters of the network model and of the training process.<br>

In [3]:
# Hilbert space cutoff
cutoff = 15

# Normalization cutoff (must be equal or smaller than cutoff dimension)
target_cutoff = 15

# Normalization weight
norm_weight = 0

# Batch size
batch_size = 8

# Number of batches (i.e. number training iterations)
num_batches = 500

# Number of state generation layers
g_depth = 1

# Number of pre-trained layers (for transfer learning)
pre_depth = 1

# Number of state classification layers
q_depth = 3

# Standard deviation of random state generation parameters
rot_sd = np.math.pi * 2
dis_sd = 0
sq_sd = 0.5
non_lin_sd = 0.5  # this is used as fixed non-linear constant.

# Standard deviation of initial trainable weights
active_sd = 0.001
passive_sd = 0.001

# Magnitude limit for trainable active parameters
clip = 1 

# Learning rate 
lr = 0.01 

# Random seeds
tf.set_random_seed(0)
rng_data = np.random.RandomState(1)

# Reset TF graph
tf.reset_default_graph()

## Variational circuits for state generation and classificaiton

### Input states: _Dataset B_

The dataset is introduced by defining the corresponding random variational circuit that generates input Gaussian and non-Gaussian states.



In [4]:
# Placeholders for class labels
batch_labels = tf.placeholder(dtype=tf.int64, shape = [batch_size])
batch_labels_fl = tf.to_float(batch_labels)

# State generation parameters
# Squeezing gate
sq_gen = tf.placeholder(dtype = tf.float32, shape = [batch_size,g_depth])

# Rotation gates        
r1_gen = tf.placeholder(dtype = tf.float32, shape = [batch_size,g_depth])
r2_gen = tf.placeholder(dtype = tf.float32, shape = [batch_size,g_depth])
r3_gen = tf.placeholder(dtype = tf.float32, shape = [batch_size,g_depth])

# Explicit definitions of the ket tensors of |0> and |1> 
np_ket0, np_ket1 = np.zeros((2, batch_size, cutoff))
np_ket0[:,0] = 1.0
np_ket1[:,1] = 1.0
ket0 = tf.constant(np_ket0, dtype = tf.float32, shape = [batch_size, cutoff])
ket1 = tf.constant(np_ket1, dtype = tf.float32, shape = [batch_size, cutoff])

# Ket of the quantum states associated to the label: i.e. |batch_labels>
ket_init = ket0 * (1.0 - tf.expand_dims(batch_labels_fl, 1)) + ket1 * tf.expand_dims(batch_labels_fl, 1)

# State generation layer
def layer_gen(i, qmode):
    
    # If label is 0 (Gaussian) prepare a coherent state with alpha=1 otherwise prepare fock |1>
    Ket(ket_init) | qmode
    Dgate((1.0 - batch_labels_fl) * 1.0, 0) | qmode
    
    # Random Gaussian operation (without displacement)
    Rgate(r1_gen[:, i]) | qmode
    Sgate(sq_gen[:, i], 0) | qmode
    Rgate(r2_gen[:, i]) | qmode

    return qmode

### Loading of pre-trained block (_Circuit A_)

We assume that _Circuit A_ has been already pre-trained (e.g. by running a dedicated Python script) and that the associated optimal weights have been saved to a NumPy file. Here we first load the such parameters and then we define _Circuit A_ as a constant pre-processing block.

In [5]:
# Loading of pre-trained weights
trained_params_npy = np.load('pre_trained/circuit_A.npy')
if trained_params_npy.shape[1] < pre_depth:
                print("Error: circuit q_depth > trained q_depth.")
                raise SystemExit(0)
            
# Convert numpy arrays to TF tensors
trained_params = tf.constant(trained_params_npy)
    
sq_pre = trained_params[0]
d_pre = trained_params[1]
r1_pre = trained_params[2]
r2_pre = trained_params[3]
r3_pre = trained_params[4]
kappa_pre = trained_params[5]

# Definition of the pre-trained Circuit A (single layer)

def layer_pre(i, qmode):
    
    # Rotation gate
    Rgate(r1_pre[i]) | qmode
    # Squeezing gate
    Sgate(tf.clip_by_value(sq_pre[i], -clip, clip), 0) 
    # Rotation gate
    Rgate(r2_pre[i]) | qmode
    # Displacement gate
    Dgate(tf.clip_by_value(d_pre[i], -clip, clip) , 0) | qmode
    # Rotation gate
    Rgate(r3_pre[i]) | qmode
    # Cubic gate
    Vgate(tf.clip_by_value(kappa_pre[i], -clip, clip) ) | qmode
    
    return qmode

### Addition of trainable layers (_Circuit B_)

As discussed in the introduction, _Circuit B_ can is obtained by adding some additional layers that we are going to train on _Dataset B_.

In [6]:
# Trainable variables
with tf.name_scope('variables'):

    # Squeeze gate
    sq_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=active_sd))

    # Displacement gate
    d_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=active_sd))

    # Rotation gates        
    r1_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=passive_sd))
    r2_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=passive_sd))
    r3_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=passive_sd))

    # Kerr gate
    kappa_var = tf.Variable(tf.random_normal(shape=[q_depth], stddev=active_sd))

    # 0-depth parameter (just to generate a gradient)
    x_var = tf.Variable(0.0)

parameters = [sq_var, d_var, r1_var, r2_var, r3_var, kappa_var]
    
# Definition of a single trainable variational layer
def layer_var(i, qmode):

    Rgate(r1_var[i]) | qmode
    Sgate(tf.clip_by_value(sq_var[i], -clip, clip), 0) | qmode
    Rgate(r2_var[i]) | qmode
    Dgate(tf.clip_by_value(d_var[i], -clip, clip) , 0) | qmode
    Rgate(r3_var[i]) | qmode
    Vgate(tf.clip_by_value(kappa_var[i], -clip, clip) ) | qmode

    return qmode

## Symbolic evaluation of the full network

We first instantiate a _StrawberryFields_ quantum simulator, taylored for simulating a single-mode quantum optical system. Then we synbolically evaluate a batch of output states.

In [7]:
prog = sf.Program(1)
eng = sf.Engine('tf', backend_options={'cutoff_dim': cutoff, 'batch_size': batch_size})
    
# Circuit B                      
with prog.context as q:
     
    # State generation network
    for k in range(g_depth):
        layer_gen(k, q[0])

    # Pre-trained network (Circuit A)
        for k in range(pre_depth):
            layer_pre(k, q[0])

    # State classification network
    for k in range(q_depth):
            layer_var(k, q[0])
                      
     # Special case q_depth==0               
    if q_depth == 0:
        Dgate(0.001, x_var ) | q[0] # almost identity operation just to generate a gradient.

# Symbolic computation of the output state
results = eng.run(prog, run_options={"eval": False})  
out_state = results.state

# Batch state norms
out_norm = tf.to_float(out_state.trace())

# Batch mean energies
mean_n = out_state.mean_photon(0)   

## Loss function, accuracy and optimizer.

As usual in machine learning, we need to define a loss function that we are going to minimize during the training phase.

As discussed in the introduction, we assume that only the vacuum state probability `p_0` is measured. Ideally, `p_0` should be large for non-Gaussian states (_label 1_), while should be small for Gaussian states (_label 0_). The circuit can be trained to this task by minimizing the _cross entropy_ loss function defined in the next cell.

Moreover, if `norm_weight` is different from zero, also a regularization term is added to the full cost function in order to reduce quantum amplitudes beyond the target Hilbert space dimension `target_cutoff`.

In [8]:
# Batch vacuum probabilities
p0 = out_state.fock_prob([0]) 

# Complementary probabilities
q0 = 1.0 - p0

# Cross entropy loss function
eps = 0.0000001
main_loss = tf.reduce_mean(-batch_labels_fl * tf.log(p0 + eps) - (1.0 - batch_labels_fl) * tf.log(q0 + eps))

# Decision function
predictions = tf.sign(p0 - 0.5) * 0.5 + 0.5

# Accuracy between predictions and labels
accuracy = tf.reduce_mean((predictions + batch_labels_fl - 1.0) ** 2)

# Norm loss. This is monitored but not minimized.
norm_loss = tf.reduce_mean((out_norm - 1.0) ** 2)

# Cutoff loss regularization. This is monitored and minimized if norm_weight is nonzero.
c_in = out_state.all_fock_probs()
cut_probs = c_in[:, :target_cutoff]
cut_norms = tf.reduce_sum(cut_probs, axis=1)
cutoff_loss = tf.reduce_mean((cut_norms - 1.0) ** 2 ) 

# Full regularized loss function
full_loss = main_loss + norm_weight * cutoff_loss

# Optimization algorithm
optim = tf.train.AdamOptimizer(learning_rate=lr)
training = optim.minimize(full_loss)   

## Training and testing

Up to now we just defined the analytic graph of the quantum network without numerically evaluating it. Now, after initializing a _TensorFlow_ session, we can finally run the actual training and testing phases. 

In [9]:
# Function generating a dictionary of random parameters for a batch of states.
def random_dict():
    param_dict = {  # Labels (0 = Gaussian, 1 = non-Gaussian)
                    batch_labels: rng_data.randint(2, size=batch_size),
        
                    # Squeezing and rotation parameters 
                    sq_gen: rng_data.uniform(low=-sq_sd, high=sq_sd, size=[batch_size, g_depth]),
                    r1_gen: rng_data.uniform(low=-rot_sd, high=rot_sd, size=[batch_size, g_depth]),
                    r2_gen: rng_data.uniform(low=-rot_sd, high=rot_sd, size=[batch_size, g_depth]),
                    r3_gen: rng_data.uniform(low=-rot_sd, high=rot_sd, size=[batch_size, g_depth]),
                 }
    return param_dict

# TensorFlow session
with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        train_loss = 0.0
        train_loss_sum = 0.0
        train_acc = 0.0
        train_acc_sum = 0.0

        test_loss = 0.0
        test_loss_sum = 0.0
        test_acc = 0.0
        test_acc_sum = 0.0

        # =========================================================
        #  	Training Phase 
        # =========================================================
        
        if q_depth > 0:
            for k in range(num_batches):
                rep_time = time.time()
                
                # Training step
                [_training,
                _full_loss,
                _accuracy,
                _norm_loss] = session.run([ training,
                                            full_loss,
                                            accuracy,
                                            norm_loss], feed_dict=random_dict())
                train_loss_sum += _full_loss
                train_acc_sum += _accuracy
                train_loss = train_loss_sum / (k + 1)
                train_acc = train_acc_sum / (k + 1)
                
                # Training log
                if ((k + 1) % 100) == 0:
                    print('Train batch: {:d}, Running loss: {:.4f}, Running acc {:.4f}, Norm loss {:.4f}, Batch time {:.4f}'
                            .format(k + 1, train_loss, train_acc, _norm_loss, time.time() - rep_time))

        # =========================================================
        #  	Testing Phase 
        # =========================================================
        num_test_batches = min(num_batches, 1000)

        for i in range(num_test_batches):  
            rep_time = time.time()
            
            # Evaluation step
            [_full_loss,
            _accuracy,
            _norm_loss,
            _cutoff_loss,
            _mean_n,
            _parameters] = session.run([full_loss,
                                        accuracy,
                                        norm_loss,
                                        cutoff_loss,
                                        mean_n,
                                        parameters], feed_dict=random_dict())
            test_loss_sum += _full_loss
            test_acc_sum += _accuracy
            test_loss = test_loss_sum / (i + 1)
            test_acc = test_acc_sum / (i + 1)
            
            # Testing log
            if ((i + 1) % 100) == 0:
                print('Test batch: {:d}, Running loss: {:.4f}, Running acc {:.4f}, Norm loss {:.4f}, Batch time {:.4f}'
                      .format(i + 1, test_loss, test_acc, _norm_loss, time.time() - rep_time))

# Compute mean photon number of the last batch of states
mean_fock = np.mean(_mean_n)
        
print('Training and testing phases completed.')
print('RESULTS:')
print('{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}'.format('train_loss', 'train_acc', 'test_loss', 'test_acc', 'norm_loss', 'mean_n'))
print('{:11f}{:11f}{:11f}{:11f}{:11f}{:11f}'.format(train_loss, train_acc, test_loss, test_acc, _norm_loss, mean_fock))      

Train batch: 100, Running loss: 0.6885, Running acc 0.3700, Norm loss 0.0460, Batch time 0.0494
Train batch: 200, Running loss: 0.6673, Running acc 0.3750, Norm loss 0.0599, Batch time 0.0498
Train batch: 300, Running loss: 0.6575, Running acc 0.3825, Norm loss 0.0495, Batch time 0.0502
Train batch: 400, Running loss: 0.6463, Running acc 0.3975, Norm loss 0.1070, Batch time 0.0497
Train batch: 500, Running loss: 0.6113, Running acc 0.4765, Norm loss 0.1387, Batch time 0.0862
Test batch: 100, Running loss: 0.4438, Running acc 0.8762, Norm loss 0.0613, Batch time 0.0206
Test batch: 200, Running loss: 0.4376, Running acc 0.8825, Norm loss 0.0801, Batch time 0.0202
Test batch: 300, Running loss: 0.4345, Running acc 0.8808, Norm loss 0.0391, Batch time 0.0199
Test batch: 400, Running loss: 0.4359, Running acc 0.8775, Norm loss 0.0698, Batch time 0.0260
Test batch: 500, Running loss: 0.4354, Running acc 0.8755, Norm loss 0.1015, Batch time 0.0256
Training and testing phases completed.
RESULT

## References

[1] Andrea Mari, Thomas R. Bromley, Josh Izaac, Maria Schuld, and Nathan Killoran.  _Transfer learning in hybrid classical-quantum neural networks_. [arXiv:xxxx.xxxx](https://arxiv.org/abs/xxxx.xxxx), (2019).

[2] Nathan Killoran, Thomas R. Bromley, Juan Miguel Arrazola, Maria Schuld, Nicolás Quesada, and Seth Lloyd. _Continuous-variable quantum neural networks_. [arXiv:1806.06871](https://arxiv.org/abs/1806.06871), (2018).

[3] Nathan Killoran, Josh Izaac, Nicolás Quesada, Ville Bergholm, Matthew Amy, and Christian Weedbrook. _Strawberry Fields: A Software Platform for Photonic Quantum Computing_. [Quantum, 3, 129 (2019)](https://doi.org/10.22331/q-2019-03-11-129).